In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from neo4j import GraphDatabase

In [2]:
df_combined = pd.read_csv("./datasets/exhibition_combined.csv")

In [3]:
df_combined

,systemNumber,Column,Value
0,O22904,accessionNumber,662 to C-1903
1,O187757,accessionNumber,C.326-1910
2,O185533,accessionNumber,683-1902
3,O162180,accessionNumber,C.240-1909
4,O151257,accessionNumber,3096-1852
...,...,...,...
54656,O333534,Depth,27 cm
54657,O1733355,Width,105 mm
54658,O1718349,Width,93 mm
54659,O308541,Height,1.9 cm


In [8]:
df_cleaned = df_combined.dropna()

triples = []
for index, row in df_cleaned.iterrows():
    subject = row['systemNumber']
    predicate = row['Column']
    object_ = row['Value']
    triples.append((subject, predicate, object_))

print(len(triples))

37089


In [9]:
# 抽样展示
# random.seed(42)
# system_numbers = list(set([item[0] for item in triples]))
# random_system_numbers = random.sample(system_numbers, 10)
# selected_triples = [item for item in triples if item[0] in random_system_numbers]

#print("随机抽取的 systemNumber:", random_system_numbers)
#  print("对应的元组:", selected_tuples)

In [10]:
# 获得三元组
# selected_triples

In [11]:
# neo4j可视化
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node_and_relationship(self, system_number, attribute, value):
        try:
            with self.driver.session() as session:
                # 使用 attribute 作为关系类型
                session.run(f"""
                    MERGE (n:Entity {{systemNumber: $system_number}})
                    MERGE (v:Value {{value: $value}})
                    MERGE (n)-[:{attribute}]->(v)
                """, system_number=system_number, value=value)
        except Exception as e:
            print(f"Error creating node and relationship: {e}")

neo4j_conn = Neo4jConnection("bolt://localhost:7687", "neo4j", "ry123456")

In [14]:
# 用neo4j打开
# for triple in selected_triples:
for triple in triples:
    neo4j_conn.create_node_and_relationship(triple[0], triple[1], triple[2])

neo4j_conn.close()

In [ ]:
# neo4j操作

MATCH (n)-[r]->(a) 

RETURN n, TYPE(r) AS relationship, properties(r), a

MATCH (n) 

DETACH DELETE n;